In [1]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.6 MB/s eta 0:00:00


In [7]:

import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="aws-0-ap-southeast-1.pooler.supabase.com",   # paste yours here
    port=5432,
    dbname="postgres",
    user="postgres.jdzitzsucntqbjvwiwxm",        # note the dot + project ref
    password="dUDQom5YQ0eVhD31",
    sslmode="require",
)
print("Connected via pooler ✅")


Connected via pooler ✅


In [8]:
query = """
SELECT *
FROM public.expense_receipt
ORDER BY created_at DESC
"""

df = pd.read_sql(query, conn)

df.head()

/tmp/ipython-input-2302208082.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,total_amount,remark,user_id,receipt_number,receipt_date,vat,withholding,discount,receipt_uuid,branch_uuid,payment_uuid,created_at,voucher_description,tax_exempt,doc_type,ref_receipt_uuid,signed_total,party_uuid,has_images
0,2000.0,,gartoonny11@gmail.com,PV6901026,2026-01-20 18:05:00,0.0,5.0,0.0,f6f2cc5b-95cb-4474-94cb-82917c6e5840,c93efb5f-07c9-4229-b6b3-568ce1c0a9ab,716d5785-207b-46e1-8a01-8909e5ca4081,2026-02-04 12:40:51.128507,ค่าเช่ารถยนต์สำนักงาน ด.01/69,0.0,RECEIPT,None,2000.0,792f2012-7864-431f-99b1-d369abeec600,True
1,20000.0,,gartoonny11@gmail.com,PV6901027,2026-01-20 18:05:00,0.0,5.0,0.0,803ab7d4-e05c-49af-91cf-70fcc6d66f06,c93efb5f-07c9-4229-b6b3-568ce1c0a9ab,716d5785-207b-46e1-8a01-8909e5ca4081,2026-02-04 12:37:41.947996,ค่าเช่าสำนักงาน ด.01/69,0.0,RECEIPT,None,20000.0,792f2012-7864-431f-99b1-d369abeec600,True
2,25500.0,,gartoonny11@gmail.com,2568-A0000003,2025-12-15 17:53:00,0.0,0.0,0.0,4fe9026f-dae0-4d07-9352-fd89d3490c82,c93efb5f-07c9-4229-b6b3-568ce1c0a9ab,e98a3376-9b5d-40f1-89be-298d5b99fcef,2026-02-02 11:29:01.770799,บริจาคเงิน โรงเรียนบ้านน้ำมี,0.0,RECEIPT,None,25500.0,b86b1f4d-09e2-4af6-869d-eac9dbe28f13,True
3,31000.0,,gartoonny11@gmail.com,RT-260123-14,2026-01-23 16:00:00,7.0,3.0,0.0,a9b7cf4d-9286-49b2-91fc-b46f8a888649,c93efb5f-07c9-4229-b6b3-568ce1c0a9ab,999df97f-ba48-4546-8ea0-a25b72340323,2026-02-02 11:15:47.810243,ค่าทำบัญชี ด.12/68,0.0,RECEIPT,None,31000.0,48b41991-e860-4210-9647-14d304e370ba,True
4,15000.0,,gartoonny11@gmail.com,RT-260117-02,2026-01-17 16:00:00,7.0,3.0,0.0,126d93ee-9c97-4319-ba92-6082f4ab9658,c93efb5f-07c9-4229-b6b3-568ce1c0a9ab,999df97f-ba48-4546-8ea0-a25b72340323,2026-02-02 11:11:39.762194,ค่าทำบัญชีเพิ่มเติม ปี 2568,0.0,RECEIPT,None,15000.0,48b41991-e860-4210-9647-14d304e370ba,True
